In [1]:
import os
import json
import glob
import shutil
from math import dist
import numpy as np
import albumentations as A
import cv2
# import matplotlib.pyplot as plt
import pandas as pd
# from matplotlib.font_manager import FontProperties

In [2]:
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']

In [3]:
kpt="../keypoint"
data_aug = "../data_aug"

In [4]:
def readKeypoints(frameKpts):
    final = []
    for i in range(0, len(frameKpts)):
        x = int(frameKpts[str(i)]["keypoint_location"][0])
        y = int(frameKpts[str(i)]["keypoint_location"][1])
        final.append((x,y))
    return final
def readVis(frameVis):
    final = []
    for i in range(0, len(frameVis)):
        final.append(int(frameVis[str(i)]["visible"]))
    return final

In [18]:
def rotate_image_without_cropping(image_path, angle, keypoints, visible):
    # 读取图像
    image = cv2.imread(image_path)
    h, w = image.shape[:2]


    # 旋转中心
    center = (w / 2, h / 2)

    # 计算旋转所需的最小画布尺寸
    radians = np.radians(angle)
    
    sin, cos = np.sin(radians), np.cos(radians)
    bound_w = int((h * abs(sin)) + (w * abs(cos)))
    bound_h = int((h * abs(cos)) + (w * abs(sin)))

    # 旋转变换
    if sin == 1.0:
        transform = A.Compose([
        # A.PadIfNeeded(min_height=bound_h, min_width=bound_w, border_mode=cv2.BORDER_REPLICATE, p=1.0),
        A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_REPLICATE, p=1.0)
        ],
        keypoint_params=A.KeypointParams(format='xy', remove_invisible=False, angle_in_degrees=True),
     p=1)

    # 应用变换
        transformed = transform(image=image, keypoints=keypoints)
        transformed_image = transformed["image"][:][210:750]
        transformed_keypoints = []
        for kpt in transformed['keypoints']:
            transformed_keypoints.append((kpt[0],kpt[1]-210))
    
        return transformed_image, transformed_keypoints

    else:
        transform = A.Compose([
            # A.PadIfNeeded(min_height=bound_h, min_width=bound_w, border_mode=cv2.BORDER_REPLICATE, p=1.0),
            A.Rotate(limit=(angle, angle), border_mode=cv2.BORDER_REPLICATE, p=1.0)
            ],
            keypoint_params=A.KeypointParams(format='xy', remove_invisible=False, angle_in_degrees=True),
         p=1)
    
        # 应用变换
        transformed = transform(image=image, keypoints=keypoints)
        transformed_image = transformed["image"]
        transformed_keypoints = transformed['keypoints']
        return transformed_image, transformed_keypoints

In [21]:
acupoints_name = ["HX6(肛門)", "HX3(尿道)", "HX1(耳中)", "TF4(神門)", "AH9(腰柢椎)", "AH11(胸椎)", "CO5(十二指腸)", "CO4(胃)", "CO3(賁門)", "CO14(肺)", "CO15(心)", "CO18(內分泌)", "HX2(輪2)", "AT2(緣中)", "LO8(扁桃體)", "LO1(牙)" , "LO2(舌)", "LO3(頷)", "LO4(垂前)", "LO5(眼)", "LO6(內耳)"]
ear_types = ["free","attached"]
angles = [-15, -10, -5,5,10, 15]
cm = {"cather":4.72, "daniel":4.74, "dominic":3.9, "jack":3.91, "jakaria":4.20, "jimmy":4.60, "vicky":4.69, "wayne":4.75,
      "gaby":4.05, "matt":5.0, "toby":4.26, "wendy":4.14, "willy":4.51}
data_aug = "../data_aug"




for angle in angles:
    for ear_type in ear_types:
        if ear_type == "free":
            acupoints_num = 21
        else:
            acupoints_num = 14
        data = os.path.join(kpt, ear_type, "4_return_to_ori_size")
        anno = os.path.join(kpt, ear_type, "2_json")
        names = os.listdir(data)
        for name in names:
            # if name == "dominic":
            for deg in degrees:
                kpt_file = os.path.join(anno, name, deg, "keypoint_location.json")
                vis_file = os.path.join(anno, name, deg, "visible.json")
                with open(kpt_file, 'r') as f1:
                    json_ = json.load(f1)
                with open(vis_file, 'r') as f2:
                    vis = json.load(f2)
                img_num = len(os.listdir(os.path.join(data, name, deg)))
                interval = img_num / 3
                idxs = [0, int(interval), int(interval*2), img_num-1]

                gt_df = pd.DataFrame(columns=list(range(acupoints_num)), index = idxs)
                
                for idx in idxs:
                    keypoints = readKeypoints(json_["frame_"+str(idx)])
                    visible = readVis(vis["frame_"+str(idx)])
                    img_path = os.path.join(data, name, deg, str(idx)+".png")
                    
                    transformed_image, transformed_keypoints = rotate_image_without_cropping(img_path, angle, keypoints, visible)
                    for j in range(0, len(transformed_keypoints)):
                        if transformed_keypoints[j][0] > 0 and transformed_keypoints[j][1] > 0:
                            gt_df.loc[idx,j] = str(transformed_keypoints[j][0])+","+str(transformed_keypoints[j][1]) +"," + str(visible[j])
                        else:
                            gt_df.loc[idx,j] = str(transformed_keypoints[j][0])+","+str(transformed_keypoints[j][1]) +"," + "0"
                
                    transformed_path = os.path.join(data_aug, ear_type, str(angle), name, deg, "img")
                    if not os.path.exists(transformed_path):
                        os.makedirs(transformed_path)

                    img_path = os.path.join(transformed_path, "test_{index}.jpg".format(index = idx))
                    cv2.imwrite(img_path, transformed_image)
                gt_df.to_csv(os.path.join(data_aug, ear_type, str(angle), name, deg, "gt.csv"))
                        # cv2.waitKey()
                        # cv2.destroyAllWindows()
            
                

    
    # for deg in degrees:
    #     angles_mean = []
    #     angles_std = []
    #     for i in range(0, 4):

    #         sum_eight_people = []

    #         for name in names:
    #             error_file = glob.glob(os.path.join(result, ear_type, name, "error",deg,"*.csv"))[0]
    #             error_df = pd.read_csv(error_file, index_col = 0)

    #             interval = len(error_df) / 3
    #             temp = list(range(len(error_df)))
    #             index = [temp[0], temp[int(interval)], temp[int(interval*2)], temp[-1]]

    #             sum_eight_people.append(error_df.iloc[index[i],:].to_numpy())

    #         sum_eight_people = np.array(sum_eight_people)

    #         angles_mean.append(np.nanmean(sum_eight_people, axis=0)*10)
    #         angles_std.append(np.nanstd(sum_eight_people, axis=0)*10)
    #     plt.figure(figsize=(15,10))

    #     for l in range(0 ,len(angles_mean)):

    #         plt.bar(np.arange(acupoints_num) + l*0.2, angles_mean[l], yerr=angles_std[l], align='center', label = '{index} degrees'.format(index = 0 + 10*l), ecolor='black', capsize=3, width=0.2)


    #     plt.ylabel('mm' , fontsize = 30, labelpad = 50, rotation = 0)

    #     plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 

    #     font_path = r'C:\Windows\Fonts\msjhl.ttc'
    #     font = FontProperties(fname=font_path, size=12)
    #     plt.xticks(np.arange(acupoints_num) + 0.3, acupoints_name[:acupoints_num], fontsize=12, rotation=-45, ha='center')


    #     plt.yticks(fontsize=30)
    #     plt.title('Localization error of different acupoints', fontsize = 30, pad = 20)

    #     plt.legend(fontsize=17)

    #     plt.tight_layout()

    #     if not os.path.isdir(os.path.join(figure, ear_type, deg)):
    #         os.makedirs(os.path.join(figure, ear_type, deg))

    #     plt.savefig(os.path.join(figure, ear_type, deg,'different acupoints.png').format(index = 0))
    #     plt.show()
        
    # [(271, 321), (214, 457), (247, 486), (279, 395), (333, 411), (369, 500), (273, 469), (284, 496), (262, 516), (264, 551), (236, 552), (211, 646), (393, 462), (295, 578), (271, 754), (233, 677), (270, 675), (307, 673), (234, 715), (269, 713), (305, 713)]
    # [(271, 321), (214, 457), (247, 486), (279, 395), (333, 411), (369, 500), (273, 469), (284, 496), (262, 516), (264, 551), (236, 552), (211, 646), (393, 462), (295, 578), (271, 754), (233, 677), (270, 675), (307, 673), (234, 715), (269, 713), (305, 713)]
        
        
                
        
        

In [15]:
import cv2
import numpy as np



# 讀取影像
image = cv2.imread(r'D:\keypoint\free\4_return_to_ori_size\cather\15cm_50mm_10deg\0.png')
# 旋轉影像
rotated_image = rotate_image_without_cropping(r'D:\keypoint\free\4_return_to_ori_size\cather\15cm_50mm_10deg\0.png', 30)
# 顯示旋轉後的影像
cv2.imshow('Rotated Image without Cropping', rotated_image)
cv2.imwrite("try.png", rotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()